In [1]:
import pandas as pd

In [2]:
# HoBo
df_oo = pd.read_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobomm.csv')

# test = pd.read_csv('/content/drive/MyDrive/ch5. 소득예측/test_oom.csv')

In [ ]:
# 함수 만들기
def label_encoder(df) :
  # 라이브러리 불러오기
  from sklearn.preprocessing import LabelEncoder
  # 인코딩할 칼럼 추출
  df_categorical = df.select_dtypes(include=['object']).columns
  # 식별자 ID 제외하기
  df_categorical = df_categorical.drop('ID')
  # 라벨 인코딩 적용
  label_encoder = LabelEncoder()
  df[df_categorical] = df[df_categorical].apply(label_encoder.fit_transform)

In [ ]:
label_encoder(df_oo)

In [ ]:
df_col = [2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 18, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
for i in df_col :
  df_oo.iloc[:,i] = df_oo.iloc[:,i].astype('category')

In [ ]:
# # # features와 label 분리 작업 + 연령대로 머신러닝
X_features = df_oo.loc[:,(df_oo.columns != 'ID') & (df_oo.columns != 'Income')]
y_label = df_oo['Income']

In [ ]:
# train_ test분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size = 0.2, random_state = 42)

In [ ]:
# categorical_features_indices = [1, 2, 3, 5, 6, 7, 8, 9,  10, 11, 12, 13, 17, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
# for i in categorical_features_indices :
#   X_train.iloc[:,i] = X_train.iloc[:,i].astype('category')
#   X_test.iloc[:,i] = X_test.iloc[:,i].astype('category')

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
import numpy as np

# RMSE 계산
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# MAE 계산
def calculate_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

# MDAPE 계산
def calculate_mdape(y_true, y_pred):
    absolute_percentage_errors = np.abs((y_true - y_pred) / y_true)
    return np.median(absolute_percentage_errors) * 100

# MAPE 계산
def calculate_mape(y_true, y_pred):
    absolute_percentage_errors = np.abs((y_true - y_pred) / y_true)
    return np.mean(absolute_percentage_errors) * 100

- LGBM

In [ ]:
from lightgbm import LGBMRegressor
import lightgbm as lgb
import numpy as np
# # 모델
# lgbm_model = LGBMRegressor()
# # 학습
# lgbm_model.fit(X_train, y_train)
# # 예측
# lgbm_pred = lgbm_model.predict(X_test)
# # 평가
# evaluation(y_test, lgbm_pred)

In [ ]:
# len(selected_features)

In [ ]:
# !pip install catboost

In [ ]:
from lightgbm import LGBMRegressor
import lightgbm as lgb
# 모델
# 모델 생성
model =  LGBMRegressor()

# Forward selection
selected_features = []
best_score = float('inf')

for i in range(X_train.shape[1]):
    scores = []
    for feature in X_train.columns:
        if feature not in selected_features:
            features_to_use = selected_features + [feature]
            model.fit(X_train.loc[:, features_to_use], y_train)
            y_pred = model.predict(X_test.loc[:, features_to_use])
            score = np.sqrt(mean_squared_error(y_test, y_pred))
            scores.append((score, feature))

    scores.sort()
    best_candidate_score, best_candidate_feature = scores[0]
    if best_candidate_score < best_score:
        selected_features.append(best_candidate_feature)
        best_score = best_candidate_score
    else:
        break

print("Selected Features:", selected_features)
print("Best RMSE:", best_score)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [ ]:
X = df_oo[selected_features]
y = df_oo['Income']

X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 모델 정의
model_lgbm = lgb.LGBMRegressor()
# 모델 학습
model_lgbm.fit(X_train1, y_train1)
# 예측
y_pred1 = model_lgbm.predict(X_test1)

# RMSE 계산
rmse = np.sqrt(mean_squared_error(y_test1, y_pred1))
# MDAPE 계산
mdape = np.median(np.abs((y_test1 - y_pred1) / y_test1)) * 100

print("RMSE:", rmse)
print("MDAPE:",mdape)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001780 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 159
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 10
[LightGBM] [Info] Start training from score 556.153500
RMSE: 583.695768299996
MDAPE: 57.90692591597385


In [ ]:
import xgboost as xgb

# 경고 메시지 출력을 숨기는 함수 정의
def suppress_xgboost_warnings():
    xgb.set_config(verbosity=0)

# suppress_xgboost_warnings 함수 호출
suppress_xgboost_warnings()

# 최적의 모델 및 점수 추적을 위한 변수 초기화
best_score = float('inf')
best_model = None
selected_features = []

# XGBoost 모델 초기화
model = xgb.XGBRegressor()

# 전진 선택법
for i in range(X_train.shape[1]):
    scores = []
    for feature in X_train.columns:
        if feature not in selected_features:
            # 선택된 특성에 현재 특성 추가
            features_to_use = selected_features + [feature]

            # 특성 선택에 따른 데이터셋 생성
            dtrain = xgb.DMatrix(X_train.loc[:, features_to_use], label=y_train, enable_categorical=True)
            dtest = xgb.DMatrix(X_test.loc[:, features_to_use], enable_categorical=True)

            # XGBoost 모델 훈련
            model = xgb.train({}, dtrain)

            # 테스트 데이터로 예측
            predictions = model.predict(dtest)

            # 평가 지표 계산
            score = calculate_rmse(y_test, predictions)
            scores.append((score, feature))

    # 특성별 점수 정렬
    scores.sort()

    # 가장 좋은 특성 선택
    best_candidate_score, best_candidate_feature = scores[0]

    # 현재까지의 최적 모델 및 점수 갱신
    if best_candidate_score < best_score:
        selected_features.append(best_candidate_feature)
        best_score = best_candidate_score
        best_model = model
    else:
        break

print("Selected Features:", selected_features)
print("Best RMSE:", best_score)

Selected Features: ['Industry_Status', 'Occupation_Status', 'Age_agegender', 'Education_Status', 'Martial_group', 'Race_yn']
Best RMSE: 591.1017962085484


In [ ]:
X = df_oo[selected_features]
y = df_oo['Income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

import xgboost as xgb

# 범주형 변수의 인덱스(X_train)
categorical_features_indices = [1,2,3,5,6,7,8,9,10,11,12,13,17,18,19,22,23,24,25,26,27,28,29,30,31]

# XGBoost 모델 초기화하기
model = xgb.XGBRegressor()

# 모델 훈련
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
params = {'categorical_feature': categorical_features_indices}
model = xgb.train(params, dtrain)

# 테스트 데이터로 예측
dtest = xgb.DMatrix(X_test, enable_categorical=True)
predictions = model.predict(dtest)

# 평가 지표 계산
rmse = calculate_rmse(y_test, predictions)
mae = calculate_mae(y_test, predictions)
mdape = calculate_mdape(y_test, predictions)
mape = calculate_mape(y_test, predictions)

print("RMSE:", rmse)
print("MAE:", mae)
print("MDAPE:", mdape)
print("MAPE:", mape)

RMSE: 591.1017962085484
MAE: 334.92361463880536
MDAPE: 58.580274309430806
MAPE: inf
